In [0]:
# Importing libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
import re
from google.colab import files
uploaded = files.upload()

Saving data_train.csv to data_train (1).csv


In [0]:
from google.colab import files
uploaded = files.upload()

Saving data_validate.csv to data_validate (1).csv


In [0]:
df = pd.read_csv("data_train.csv")
df = df.head(10000)
dev = pd.read_csv("data_validate.csv")

#comments = Airplane.Summary.dropna().str.lower().str.split().reset_index(drop=True)
comments = df.comments.str.lower().reset_index(drop=True)
string = ""
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+#'''
for i in range(len(comments)):
    new_comments = ""
    for char in comments[i]:
        if char not in punctuations:
            new_comments = new_comments + char
    comments[i] = re.sub(r'[0-9]'," ", new_comments)
for i in range(len(comments)):
    tokens = []
    for word in comments[i].split():
        if word not in STOP_WORDS:
            tokens.append(word)
    comments[i] = tokens
    string = string+" ".join(comments[i])
#print(comments)
count = comments.apply(pd.value_counts).fillna(0)
count.index += 1
count = count.set_index('comment'+count.index.astype(str))
count.columns.names = ['word']

##############################################################
dev_comments = dev.comments.str.lower().reset_index(drop=True)
string = ""
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+#'''
for i in range(len(dev_comments)):
    dev_new_comments = ""
    for char in dev_comments[i]:
        if char not in punctuations:
            dev_new_comments = dev_new_comments + char
    dev_comments[i] = re.sub(r'[0-9]'," ", dev_new_comments)
for i in range(len(dev_comments)):
    tokens = []
    for word in dev_comments[i].split():
        if word not in STOP_WORDS:
            tokens.append(word)
    dev_comments[i] = tokens
    string = string+" ".join(dev_comments[i])
#print(comments)
dev_count = dev_comments.apply(pd.value_counts).fillna(0)
dev_count.index += 1
dev_count = dev_count.set_index('comment'+dev_count.index.astype(str))
#for ele in dev_count.columns:
#    if ele not in count.columns:
dev_count = dev_count.drop(columns=list(set(dev_count.columns)-set(count.columns)))

for col in list(set(count.columns)-set(dev_count.columns)):
    dev_count[col] = 0
count = count.reindex(sorted(count.columns), axis=1)
dev_count = dev_count.reindex(sorted(dev_count.columns), axis=1)
print(dev_count.shape)
print(count.shape)


# Hyper Parameters
input_size = count.shape[1]
num_classes = 2
num_epochs = 5
batch_size = 100
learning_rate = 0.001




x_data = Variable(torch.Tensor(count.values))
y_data = Variable(torch.LongTensor(df.none))
labels = Variable(torch.LongTensor(dev.none))


# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out


model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training the Model
for epoch in range(num_epochs):
    model.train()

    # Forward + Backward + Optimize
    optimizer.zero_grad()
    y_predict = model(x_data)
    loss = criterion(y_predict, y_data)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
        print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f'
               % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
"""
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))
"""
correct = 0
total = dev_count.shape[0]
new_x = Variable(torch.Tensor(dev_count.values))
y_pred = model(new_x)


y_pred = torch.argmax(y_pred, dim=1)

#correct = (y_pred == labels).sum()/len(labels)
#a1 = torch.tensor([1, 1])
#a2 = torch.tensor([0, 1])
for i in range(len(labels)):
    if y_pred.tolist()[i]==labels[i]:
        correct =correct+1
print(correct/len(labels))


(9245, 37021)
(10000, 37021)
0.541157382368848
